<a href="https://colab.research.google.com/github/hm-sharma/RedBus-Hackathon_project/blob/main/Redbus_Hackathon(h_m).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
transactions=pd.read_csv('/content/transactions.csv')

In [51]:
transactions.head(10)

,doj,doi,srcid,destid,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd
0,2023-03-01,2023-01-30,45,46,Karnataka,Tamil Nadu,Tier 1,Tier 1,8.0,76.0,30
1,2023-03-01,2023-01-30,46,45,Tamil Nadu,Karnataka,Tier 1,Tier 1,8.0,70.0,30
2,2023-03-01,2023-01-30,45,47,Karnataka,Andhra Pradesh,Tier 1,Tier 1,4.0,142.0,30
3,2023-03-01,2023-01-30,47,45,Andhra Pradesh,Karnataka,Tier 1,Tier 1,0.0,68.0,30
4,2023-03-01,2023-01-30,46,9,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,9.0,162.0,30
5,2023-03-01,2023-01-30,9,46,Tamil Nadu,Tamil Nadu,Tier2,Tier 1,0.0,63.0,30
6,2023-03-01,2023-01-30,46,48,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,0.0,309.0,30
7,2023-03-01,2023-01-30,48,46,Tamil Nadu,Tamil Nadu,Tier2,Tier 1,0.0,165.0,30
8,2023-03-01,2023-01-30,45,9,Karnataka,Tamil Nadu,Tier 1,Tier2,3.0,84.0,30
9,2023-03-01,2023-01-30,9,45,Tamil Nadu,Karnataka,Tier2,Tier 1,0.0,39.0,30


In [52]:
df_test=pd.read_csv('/content/test_8gqdJqH.csv')

In [53]:
df_test.head()

,route_key,doj,srcid,destid
0,2025-02-11_46_45,2025-02-11,46,45
1,2025-01-20_17_23,2025-01-20,17,23
2,2025-01-08_02_14,2025-01-08,2,14
3,2025-01-08_08_47,2025-01-08,8,47
4,2025-01-08_09_46,2025-01-08,9,46


In [54]:
train=pd.read_csv('/content/train.csv')

In [55]:
train.head()

,doj,srcid,destid,final_seatcount
0,2023-03-01,45,46,2838.0
1,2023-03-01,46,45,2298.0
2,2023-03-01,45,47,2720.0
3,2023-03-01,47,45,2580.0
4,2023-03-01,46,9,4185.0


To train the model we'll need these features.


In [56]:

# Filter transactions to only include records where days_before_departure >= 15
transactions_filtered = transactions[transactions['dbd'] >= 15].copy()

# Aggregate latest city-level info as of 15 days before journey
route_city_features = transactions_filtered.groupby(['srcid', 'destid', 'doj']).agg({
    'srcid_region': 'last',
    'destid_region': 'last',
    'srcid_tier': 'last',
    'destid_tier': 'last'
}).reset_index()


In [57]:
train['doj'] = pd.to_datetime(train['doj'])

train['day_of_week'] = train['doj'].dt.dayofweek  # Monday=0
train['month'] = train['doj'].dt.month

from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=train['doj'].min(), end=train['doj'].max())
train['is_holiday'] = train['doj'].isin(holidays).astype(int)

train['is_weekend'] = train['day_of_week'].isin([5, 6]).astype(int)  # Saturday=5, Sunday=6


In [58]:
# Aggregate max cumulative seats and searches up to 15 days before journey
agg_features = transactions_filtered.groupby(['srcid', 'destid', 'doj']).agg({
    'cumsum_seatcount': 'max',
    'cumsum_searchcount': 'max'
}).reset_index()

# Helper function to get cumulative values at specific days before journey
def get_cumulative_at_day(df, day):
    return df[df['dbd'] == day][['srcid', 'destid', 'doj', 'cumsum_seatcount', 'cumsum_searchcount']]

cum_15 = get_cumulative_at_day(transactions, 15)
cum_22 = get_cumulative_at_day(transactions, 22)

# Calculate booking and search velocity between day 22 and day 15
velocity = pd.merge(cum_15, cum_22, on=['srcid', 'destid', 'doj'], suffixes=('_15', '_22'))
velocity['booking_velocity_7d'] = velocity['cumsum_seatcount_15'] - velocity['cumsum_seatcount_22']
velocity['search_velocity_7d'] = velocity['cumsum_searchcount_15'] - velocity['cumsum_searchcount_22']
# Merge velocity with aggregated features
features = pd.merge(agg_features, velocity[['srcid', 'destid', 'doj', 'booking_velocity_7d', 'search_velocity_7d']], on=['srcid', 'destid', 'doj'], how='left')

# Calculate search to booking ratio, fill NaN with 0
features['search_to_booking_ratio'] = features['cumsum_seatcount'] / features['cumsum_searchcount']
features['search_to_booking_ratio'] = features['search_to_booking_ratio'].fillna(0)


In [59]:
# Convert date_of_journey to datetime
train['doj'] = pd.to_datetime(train['doj'], errors='coerce')

# Aggregate average and median demand per route
route_stats = train.groupby(['srcid', 'destid']).agg(
    avg_historical_demand=('final_seatcount', 'mean'),
    median_historical_demand=('final_seatcount', 'median'),
    route_popularity=('final_seatcount', 'count')
).reset_index()

# Average demand by day of week per route
dow_stats = train.groupby(['srcid', 'destid', train['doj'].dt.dayofweek]).agg(
    avg_demand_dow=('final_seatcount', 'mean')
).reset_index().rename(columns={'doj': 'day_of_week'})

# Add day_of_week to train
train['day_of_week'] = train['doj'].dt.dayofweek

# Merge route_stats and dow_stats with train
train = train.merge(route_stats, on=['srcid', 'destid'], how='left')
train = train.merge(dow_stats, on=['srcid', 'destid', 'day_of_week'], how='left')


In [60]:
# Using the is_holiday flag already created in train
holiday_demand = train.groupby(['srcid', 'destid', 'is_holiday']).agg(
    avg_demand=('final_seatcount', 'mean')
).reset_index()

# Separate holiday and non-holiday averages
holiday_avg = holiday_demand[holiday_demand['is_holiday'] == 1][['srcid', 'destid', 'avg_demand']].rename(columns={'avg_demand': 'holiday_avg'})
non_holiday_avg = holiday_demand[holiday_demand['is_holiday'] == 0][['srcid', 'destid', 'avg_demand']].rename(columns={'avg_demand': 'non_holiday_avg'})

# Merge and compute boost ratio
holiday_boost = pd.merge(holiday_avg, non_holiday_avg, on=['srcid', 'destid'])
holiday_boost['holiday_demand_boost'] = holiday_boost['holiday_avg'] / holiday_boost['non_holiday_avg']

# Merge with train
train = train.merge(holiday_boost[['srcid', 'destid', 'holiday_demand_boost']], on=['srcid', 'destid'], how='left')


In [61]:
train = train.sort_values(['srcid', 'destid', 'doj'])
train['doj_gap'] = train.groupby(['srcid', 'destid'])['doj'].diff().dt.days


In [62]:

from functools import reduce

def merge_feature_dataframes(route_city_features, features, train):
    train_subset = train[[
        'srcid', 'destid', 'doj',
        'avg_historical_demand', 'median_historical_demand', 'avg_demand_dow',
        'route_popularity', 'holiday_demand_boost', 'final_seatcount'
    ]]

    df_list = [route_city_features, features, train_subset]

    final_train_df = reduce(
        lambda left, right: pd.merge(left, right, on=['srcid', 'destid', 'doj'], how='left'),
        df_list
    )

    return final_train_df


# Usage example (assuming the dataframes are already prepared):
# final_train_df = merge_feature_dataframes(route_city_features, features, train)
# print(final_train_df.head())


In [64]:
route_city_features['doj'] = pd.to_datetime(route_city_features['doj'])
features['doj'] = pd.to_datetime(features['doj'])
train['doj'] = pd.to_datetime(train['doj'])

final_train_df = merge_feature_dataframes(route_city_features, features, train)
print(final_train_df.head())

   srcid  destid        doj         srcid_region        destid_region  \
0      1       2 2023-03-01  Maharashtra and Goa  Maharashtra and Goa   
1      1       2 2023-03-02  Maharashtra and Goa  Maharashtra and Goa   
2      1       2 2023-03-03  Maharashtra and Goa  Maharashtra and Goa   
3      1       2 2023-03-04  Maharashtra and Goa  Maharashtra and Goa   
4      1       2 2023-03-05  Maharashtra and Goa  Maharashtra and Goa   

  srcid_tier destid_tier  cumsum_seatcount  cumsum_searchcount  \
0      Tier2      Tier 1               0.0                90.0   
1      Tier2      Tier 1               0.0                75.0   
2      Tier2      Tier 1               0.0                75.0   
3      Tier2      Tier 1               0.0                65.0   
4      Tier2      Tier 1               0.0                80.0   

   booking_velocity_7d  search_velocity_7d  search_to_booking_ratio  \
0                  0.0                35.0                      0.0   
1                  0.0

In [65]:
final_train_df

,srcid,destid,doj,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,booking_velocity_7d,search_velocity_7d,search_to_booking_ratio,avg_historical_demand,median_historical_demand,avg_demand_dow,route_popularity,holiday_demand_boost,final_seatcount
0,1,2,2023-03-01,Maharashtra and Goa,Maharashtra and Goa,Tier2,Tier 1,0.0,90.0,0.0,35.0,0.000000,1304.010417,1282.5,1201.979167,672.0,1.016572,1115.0
1,1,2,2023-03-02,Maharashtra and Goa,Maharashtra and Goa,Tier2,Tier 1,0.0,75.0,0.0,15.0,0.000000,1304.010417,1282.5,1153.958333,672.0,1.016572,1040.0
2,1,2,2023-03-03,Maharashtra and Goa,Maharashtra and Goa,Tier2,Tier 1,0.0,75.0,0.0,30.0,0.000000,1304.010417,1282.5,1271.510417,672.0,1.016572,935.0
3,1,2,2023-03-04,Maharashtra and Goa,Maharashtra and Goa,Tier2,Tier 1,0.0,65.0,0.0,30.0,0.000000,1304.010417,1282.5,1278.906250,672.0,1.016572,1170.0
4,1,2,2023-03-05,Maharashtra and Goa,Maharashtra and Goa,Tier2,Tier 1,0.0,80.0,0.0,35.0,0.000000,1304.010417,1282.5,1572.760417,672.0,1.016572,1480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73095,48,46,2025-02-24,Tamil Nadu,Tamil Nadu,Tier2,Tier 1,51.0,2517.0,21.0,1500.0,0.020262,NaN,NaN,NaN,NaN,NaN,NaN
73096,48,46,2025-02-25,Tamil Nadu,Tamil Nadu,Tier2,Tier 1,27.0,2094.0,21.0,1263.0,0.012894,NaN,NaN,NaN,NaN,NaN,NaN
73097,48,46,2025-02-26,Tamil Nadu,Tamil Nadu,Tier2,Tier 1,84.0,3432.0,51.0,1779.0,0.024476,NaN,NaN,NaN,NaN,NaN,NaN
73098,48,46,2025-02-27,Tamil Nadu,Tamil Nadu,Tier2,Tier 1,396.0,11871.0,195.0,6024.0,0.033359,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
print(final_train_df.isna().sum().sum())


35400
